In [1]:
import numpy
from matplotlib import pyplot
%matplotlib inline

from scipy import linalg

In [2]:
# set the font family and size to use for Matplotlib figures.
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [3]:
def btcs_implicit_2d(T0, nt, dt, delta, alpha, Tb):
    ny, nx = T0.shape
    M, N = nx - 2, ny - 2
    
    sigma = alpha * dt / delta**2
    
    A = lhs_operator(M, N, sigma)
    
    T = T0[1:-1, 1:-1].flatten()
    I, J = int(M / 2), int(N / 2)
    
    for n in range(nt):
        b = rhs_vector(T, M, N, sigma, Tb)
        T = linalg.solve(A, b)
        if T[J * M + I] >= 70.0:
            break
    
    print('time step: {} - center at T={:.1f} at t={:.2f} s'.format(n + 1, T[J, I], 
                                                                   (n + 1) * dt))
    return map_1d_to_2d(T, nx, ny, Tb)

In [4]:
def map_1d_to_2d(T1, nx, ny, Tb):

    T = numpy.zeros((ny, nx))
    T[1:-1, 1:-1] = T1.reshape((ny - 2, nx - 2))
    
    # Dirichlet conditions
    T[:, 0] = Tb  # left
    T[0, :] = Tb  # bottom
    
    # Neumann conditions
    T[:, -1] = T[:, -2]  # top
    T[-1, :] = T[-2, :]  # right
    
    return T

In [5]:
def rhs_vector(T, M, N, sigma, Tb):
    
    b = T / sigma
    
    for j in range(N):
        for i in range(M):
            I = j * M + i
            if i == 0:
                b[I] += Tb
            if j == 0:
                b[I] += Tb
    
    return b

In [6]:
def lhs_operator(M, N, sigma):
    
    A = numpy.zeros((M * N, M * N))
    
    for j in range(N):
        for i in range(M):
            I = j * M + i  # row idx
            IS = I - M
            IW = I - 1
            IE = I + 1
            IN = I + M
            
            ## side points
            # left
            if i == 0:
                A[I, I] = 1.0 / sigma + 4.0
                A[I, IS] = -1.0
                A[I, IE] = -1.0
                A[I, IN] = -1.0
            # bottom
            elif j == 0:
                A[I, I] = 1.0 / sigma + 4.0
                A[I, IW] = -1.0
                A[I, IE] = -1.0
                A[I, IN] = -1.0
            # right
            elif i == M - 1:
                A[I, I] = 1.0 / sigma + 3.0
                A[I, IS] = -1.0
                A[I, IW] = -1.0
                A[I, IN] = -1.0
            # top
            elif i == N - 1:
                A[I, I] = 1.0 / sigma + 3.0
                A[I, IS] = -1.0
                A[I, IW] = -1.0
                A[I, IE] = -1.0
        
            ## corner points
            # bottom left
            elif i == 0 and j == 0:
                A[I, I] = 1.0 / sigma + 4.0
                A[I, IS] = -1.0
                A[I, IE] = -1.0
            # bottom right
            elif i == M - 1 and j == 0:
                A[I, I] = 1.0 / sigma + 3.0
                A[I, IW] = -1.0
                A[I, IN] = -1.0
            # top left
            elif i == 0 and j == N - 1:
                A[I, I] = 1.0 / sigma + 3.0
                A[I, IS] = -1.0
                A[I, IE] = -1.0
            # top right
            elif i == M - 1 and j == N - 1:
                A[I, I] = 1.0 / sigma + 2.0
                A[I, IS] = -1.0
                A[I, IW] = -1.0
            
            ## interior points
            else:
                A[I, I] = 1.0 / sigma + 4.0
                A[I, IS] = -1.0
                A[I, IW] = -1.0
                A[I, IE] = -1.0
                A[I, IN] = -1.0
    return A

In [7]:
# set params
Lx = 0.01  # length of the plate in the x direction
Ly = 0.01  # height of the plate in the y direction
nx = 21  # number of points in the x direction
ny = 21  # number of points in the y direction
dx = Lx / (nx - 1)  # grid spacing in the x direction
dy = Ly / (ny - 1)  # grid spacing in the y direction
alpha = 1e-4  # thermal diffusivity of the plate

# define the grid positions
x = numpy.linspace(0.0, Lx, num=nx)
y = numpy.linspace(0.0, Ly, num=ny)

# initial conditions
Tb = 100.0  # temperature at the left and bottom boundaries
T0 = 20.0 * numpy.ones((ny, nx))
T0[0, :] = Tb
T0[:, 0] = Tb

In [14]:
# set time step size based on CFL limits
sigma = 0.5
dt = sigma / (1 / dx**2 + 1 / dy**2) / alpha  # time-step size
nt = 300  # number of time steps to compute

T = btcs_implicit_2d(T0, nt, dt, dx, dy, alpha)

IndexError: index 361 is out of bounds for axis 1 with size 361